In [1]:
import tensorflow as tf
from tensorflow import keras


In [2]:
print(tf.keras.__version__)
print(tf.__version__)

2.2.4-tf
2.0.0


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt

from sklearn import preprocessing


In [4]:

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

In [5]:
directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#print(turbo.values)


In [6]:
# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1,2,3,4], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

In [7]:
# Get the cycles and useful sensors. 
turboUsefulSens = turboSens.drop([0, 9, 17, 18], axis=1)

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

In [9]:
turboCycles = turbo[1]
#print(turboCycles)

In [10]:
deteriorate = []
maxCycle = float(turboCycles.max())


In [11]:
currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1


In [12]:
deteriorate = pd.DataFrame(deteriorate)

In [13]:
deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


In [14]:
norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')

#print(deteriorate.head(5))
deteriorate = deteriorate.sample(frac=1)
#print(deteriorate.tail())

deteriorate.columns = getColumns(deteriorate.shape[1])

In [15]:
norm_sens = deteriorate.drop(0, axis=1)
norm_sens = norm_sens.to_numpy()
deteriorate = deteriorate[0].to_numpy()

In [16]:
#split = round((len(deteriorate) / 5) * 4)

#detTrain = deteriorate[:split]
#detTest = deteriorate[split:]

#sensTrain = norm_sens[:split]
#sensTest = norm_sens[split:]

#columnCount = sensTrain.shape[1]

In [17]:
#model = tf.keras.models.Sequential([
#    tf.keras.layers.Dense(64, activation='relu'),
#    tf.keras.layers.Dense(64, activation='relu'),
#    tf.keras.layers.Dense(1)
#])

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [18]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mae', 
              metrics=['mae', 'mse'])
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True),
#              loss='mse', 
#              metrics=['mae', 'mse'])

In [ ]:
model.fit(norm_sens, deteriorate, batch_size=64, epochs=10000, validation_split = 0.2, verbose=1)

Train on 16504 samples, validate on 4127 samples
Epoch 1/1000
16504/16504 [==============================] - 1s 73us/sample - loss: 27.3129 - mae: 27.3129 - mse: 1639.8159 - val_loss: 29.2826 - val_mae: 29.2826 - val_mse: 1842.2314
Epoch 2/1000
16504/16504 [==============================] - 1s 78us/sample - loss: 27.2575 - mae: 27.2575 - mse: 1633.4945 - val_loss: 29.4230 - val_mae: 29.4230 - val_mse: 1869.0204
Epoch 3/1000
16504/16504 [==============================] - 1s 88us/sample - loss: 27.2966 - mae: 27.2966 - mse: 1640.0717 - val_loss: 28.9688 - val_mae: 28.9688 - val_mse: 1782.2389
Epoch 4/1000
16504/16504 [==============================] - 1s 77us/sample - loss: 27.3144 - mae: 27.3144 - mse: 1640.3357 - val_loss: 29.1621 - val_mae: 29.1621 - val_mse: 1784.6571
Epoch 5/1000
16504/16504 [==============================] - 1s 85us/sample - loss: 27.3052 - mae: 27.3052 - mse: 1639.3862 - val_loss: 29.1229 - val_mae: 29.1229 - val_mse: 1787.4656
Epoch 6/1000
16504/16504 [==========